In [15]:
# Importing the libraries needed
import os
import re
import sys
from string import punctuation
sys.path.append("/Users/leonardovida/dev/HistAware")

import pandas as pd
import numpy as np

import nltk
import nltk.data
from nltk.sentiment import vader
from nltk.corpus import stopwords

from gensim import corpora
from gensim import models
import enchant
from enchant.checker import SpellChecker
from tqdm import tqdm_notebook as tqdm

from src import iterators

In [4]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
# Retrieved saved files
csv = iterators.iterate_directory("../data/processed/selected_articles/", ".csv")
df = pd.concat([pd.read_csv(c["article_path"]) for c in csv],ignore_index=True)
df.sort_values(by=["count"], ascending=False, inplace=True)

You are interested in `df["text"]` (or df.text). Which is the column that contains all the selected texts.

What we would like to do is to select similar texts based on a golden sample of texts. 
- Which means that we will go through all our dataset and select based on the frequency of the appearance of certain words (oil, gas and coal)
- After that the results are ranked based on the number of matches of these words and these synonyms.
- Once that is done, you have the `df` at hand in this notebook.

You can assume that the first 100 rows of this dataframe are the "golden" labelled text representing texts we want to find similar texts to.

In [8]:
df_selected = df[0:100]
df_remaining = df[101:]

I suggest you to look at the examples every time you want to "understand the score" instead of looking only at the similarity score you will get. If you have time you could go through the first 100 rows and create a even better selection.... :)

### A small start

In [10]:
# Let's create a list of text, just to experiment with the text
word_nl=[]
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    word_nl.append(row["text"])

Cleaning the list of words

In [16]:
word_nl=["".join([l for l in word if l not in punctuation]) for word in word_nl]  #remove punctuation
word_nl=[word.lower() for word in word_nl]  # convert to lower case
word_nl=[" ".join(word.split()) for word in word_nl]   # remove double spaces by splitting the strings into words and joining these words again
word_nl=[re.sub(r'[^a-zA-z\s]', '', word) for word in word_nl]  # to remove special characters and symbols

These are not always necessary! We are losing "meaning". However, on the types of analyses you will do these steps might be useful. Try with and without!


From here apply the tf-idf/BM 25 or other similarity algorithms!
- Use the documentation (and internent examples) from the following libraries:
    - nltk --> for the already-written-out main algorithms. This will be your main library with:
    - gensim
    - Enchant (might be useful)